# 01 - Ingesta de Datos de Brokers

Este notebook procesa las novedades de cuentas corrientes de los diferentes brokers.

**Proceso:**
1. Lee archivos de `data/01_raw/cliente_X/`
2. Aplica el Strategy Pattern según el broker
3. Normaliza los datos
4. Actualiza históricos en `data/02_processed/accounts/`

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Agregar src al path
sys.path.append(str(Path.cwd().parent / 'src'))

from connectors import BullMarketStrategy, BalanzStrategy
from core.account_manager import AccountManager
from utils.file_io import load_yaml_config, find_latest_file

In [ ]:
# Configuración de rutas
BASE_PATH = Path.cwd().parent
CONFIG_PATH = BASE_PATH / 'config'
DATA_PATH = BASE_PATH / 'data'
RAW_PATH = DATA_PATH / '01_raw'
PROCESSED_PATH = DATA_PATH / '02_processed'

# Cargar configuración de clientes
clients_config = load_yaml_config(CONFIG_PATH / 'clients_config.yaml')
clients = [c for c in clients_config['clients'] if c['active']]

print(f"Clientes activos: {len(clients)}")

In [ ]:
# Inicializar Account Manager
account_mgr = AccountManager(PROCESSED_PATH)

# Inicializar estrategias de brokers
strategies = {
    'bull_market': BullMarketStrategy(),
    'balanz': BalanzStrategy()
}

In [ ]:
# Procesar cada cliente
for client in clients:
    client_id = client['id']
    broker = client['broker']
    source_folder = RAW_PATH / client['source_folder']
    
    print(f"\n{'='*60}")
    print(f"Procesando: {client['name']} ({client_id})")
    print(f"Broker: {broker}")
    print(f"{'='*60}")
    
    # Buscar último archivo
    latest_file = find_latest_file(source_folder, '*.xlsx')
    
    if latest_file is None:
        print(f"⚠️  No se encontraron archivos en {source_folder}")
        continue
    
    print(f"Archivo: {latest_file.name}")
    
    # Aplicar estrategia del broker
    strategy = strategies[broker]
    
    try:
        # Leer y normalizar datos
        new_transactions = strategy.read_cuenta_corriente(latest_file)
        
        print(f"Transacciones nuevas: {len(new_transactions)}")
        
        # Actualizar histórico
        account_mgr.update_account(client_id, new_transactions)
        
        # Mostrar resumen
        summary = account_mgr.get_account_summary(client_id)
        print(f"\nResumen:")
        for key, value in summary.items():
            print(f"  {key}: {value}")
        
    except Exception as e:
        print(f"✗ Error procesando {client_id}: {e}")

In [ ]:
print("\n✓ Proceso completado")